In [ ]:
import os
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np
from ResNet import ResidualUnit
from Inception import InceptionUnit
from Xception import XceptionUnit

In [ ]:
model_resnet = keras.models.Sequential()
model_resnet.add(keras.layers.Conv2D(8, 7, strides = 2, input_shape = [224, 224, 3], padding = "same", use_bias = False))
model_resnet.add(keras.layers.BatchNormalization())
model_resnet.add(keras.layers.Activation("relu"))
model_resnet.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding = "same"))
prev_filters = 8
for filters in [8, 16, 32]:
    strides = 1 if filters == prev_filters else 2
    model_resnet.add(ResidualUnit(filters, strides = strides))
    prev_filters = filters
model_resnet.add(keras.layers.GlobalAveragePooling2D())
model_resnet.add(keras.layers.Flatten())
model_resnet.add(keras.layers.Dense(10, activation = "relu"))
model_resnet.add(keras.layers.Dense(1, activation = "sigmoid"))

In [ ]:
model_googlenet = keras.models.Sequential()
model_googlenet.add(keras.layers.Conv2D(64, 7, strides = 2, input_shape = [224, 224, 3], padding = "same", use_bias = False))
model_googlenet.add(keras.layers.BatchNormalization())
model_googlenet.add(keras.layers.Activation("relu"))
model_googlenet.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding = "same"))
model_googlenet.add(InceptionUnit())
model_googlenet.add(keras.layers.GlobalAveragePooling2D())
model_googlenet.add(keras.layers.Flatten())
model_googlenet.add(keras.layers.Dense(10, activation = "relu"))
model_googlenet.add(keras.layers.Dense(1, activation = "sigmoid"))

In [ ]:
model_xception = keras.models.Sequential()
model_xception.add(keras.layers.Conv2D(8, 7, strides = 2, input_shape = [224, 224, 3], padding = "same", use_bias = False))
model_xception.add(keras.layers.BatchNormalization())
model_xception.add(keras.layers.Activation("relu"))
model_xception.add(XceptionUnit(8, isEntryExit=True))
model_xception.add(XceptionUnit(16))
model_xception.add(XceptionUnit(16))
model_xception.add(XceptionUnit(8, isEntryExit=True))
model_xception.add(keras.layers.SeparableConv2D(8, 3, padding = "same"))
model_xception.add(keras.layers.BatchNormalization())
model_xception.add(keras.layers.Activation("relu"))
model_xception.add(keras.layers.Flatten())
model_xception.add(keras.layers.Dense(10, activation = "relu"))
model_xception.add(keras.layers.Dense(1, activation = "softmax"))

In [ ]:
img_height = 224
img_weight = 224
batch_size = 16

In [ ]:
ds_train = keras.utils.image_dataset_from_directory(
    "D:\Programming\Python\Tender-Notice-Extraction\Datasets",
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (img_height, img_weight),
    shuffle = True,
    seed = 123,
    validation_split = 0.20,
    subset = "training"
)

In [ ]:
ds_valid = keras.utils.image_dataset_from_directory(
    "D:\Programming\Python\Tender-Notice-Extraction\Datasets",
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (img_height, img_weight),
    shuffle = True,
    seed = 123,
    validation_split = 0.20,
    subset = "validation"
)

In [ ]:
class_names = ds_train.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
ds_take = ds_train.take(1)
for images, labels in ds_take:
  print(labels)
  # print(images[0].shape)
  # plt.imshow(images[0]/255)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    # plt.imshow(images[i].numpy().astype("uint8"))
    plt.imshow(images[i]/255)
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in ds_train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
# ds_train = ds_train.map(lambda x, y: (normalization_layer(x), y))
# image_batch_train, labels_batch_train = next(iter(ds_train))
# first_image = image_batch_train[0]
# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

In [ ]:
# ds_valid = ds_valid.map(lambda x, y: (normalization_layer(x), y))
# image_batch_valid, labels_batch_valid = next(iter(ds_valid))
# first_image = image_batch_valid[0]
# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [ ]:
ds_train = ds_train.map(process)
ds_valid = ds_valid.map(process)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
  print(labels)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# NAME = "tender-cnn1-{}".format(int(time.time()))
# tensorboard = keras.callbacks.TensorBoard(log_dir=f"logs/Xception{NAME}", histogram_freq=0, write_graph=True, write_images=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# checkpoint_cb = keras.callbacks.ModelCheckpoint(f"models/Xception{NAME}.h5", save_best_only=True)

In [ ]:

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model_xception.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history = model_xception.fit(ds_train,
                    epochs=20,
                    validation_data=ds_valid,
                    callbacks=[early_stopping_cb])
                    
                   